In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import losses
from tensorflow.keras import preprocessing

### load and preprocess data

In [2]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'C50/C50train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 2500 files belonging to 50 classes.
Using 2000 files for training.


### create a validation and test dataset

In [3]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'C50/C50train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 2500 files belonging to 50 classes.
Using 500 files for validation.


In [4]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'C50/C50test', 
    batch_size=batch_size)

Found 2500 files belonging to 50 classes.


### Prepare dataset for training

In [5]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')

In [6]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [7]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

### Apply TextVectorization Layer

In [8]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [9]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

### Performance

In [10]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model

In [23]:
embedding_dim = 64
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(50,activation='softmax')])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          640064    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 64)          0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 64)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                3250      
Total params: 643,314
Trainable params: 643,314
Non-trainable params: 0
_________________________________________________________________


### Loss function and optimizer

In [24]:
model.compile(
    loss= SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])


### Train the model

In [25]:
tf.autograph.experimental.do_not_convert(
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=50)
)

Epoch 1/50
63/63 [==============================] - 1s 7ms/step - loss: 3.9069 - accuracy: 0.0950 - val_loss: 3.9005 - val_accuracy: 0.2060
Epoch 2/50
63/63 [==============================] - 0s 7ms/step - loss: 3.8830 - accuracy: 0.3880 - val_loss: 3.8803 - val_accuracy: 0.3220
Epoch 3/50
63/63 [==============================] - 0s 6ms/step - loss: 3.8438 - accuracy: 0.5210 - val_loss: 3.8438 - val_accuracy: 0.3840
Epoch 4/50
63/63 [==============================] - 0s 6ms/step - loss: 3.7797 - accuracy: 0.5745 - val_loss: 3.7856 - val_accuracy: 0.4100
Epoch 5/50
63/63 [==============================] - 0s 6ms/step - loss: 3.6850 - accuracy: 0.6030 - val_loss: 3.7025 - val_accuracy: 0.4300
Epoch 6/50
63/63 [==============================] - 0s 6ms/step - loss: 3.5632 - accuracy: 0.6355 - val_loss: 3.5966 - val_accuracy: 0.4600
Epoch 7/50
63/63 [==============================] - 0s 5ms/step - loss: 3.4141 - accuracy: 0.6555 - val_loss: 3.4732 - val_accuracy: 0.4780
Epoch 8/50
63/63 [==

<function tensorflow.python.autograph.impl.api.do_not_convert.<locals>.wrapper(*args, **kwargs)>

### Evaluate the Model

In [28]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

79/79 [==============================] - 0s 745us/step - loss: 1.4666 - accuracy: 0.6136
Loss:  1.466578483581543
Accuracy:  0.6136000156402588


### Accuracy over time

In [29]:
history_dict = history.history
history_dict.keys()

NameError: name 'history' is not defined

In [30]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

NameError: name 'history_dict' is not defined